In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 1, 26)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market'.split()
colt = 'trans name qty target active spd current change percent'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market
0,28,S,MCS,"5,000",14.60,2,CUT,SET


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
1,B,EPG,-5,RD10pct,SET100,"6,000",9.90,10.30,-0.30,-2.83%,2,Buy,"59,400.00"
10,B,SCC,-6,RD05pct,SET50,300,374.00,380.00,2.00,+0.53%,2,Buy,"112,200.00"
11,B,STA,-2,RD15pct,SET100,"2,500",28.00,28.50,-0.50,-1.72%,2,Buy,"70,000.00"
22,S,NER,4,CUT,SET,"9,000",7.30,7.10,0.00,0.00%,2,Sell,"65,700.00"
25,S,TSTH,10,LDH,SET,"75,000",1.48,1.38,0.01,+0.73%,2,Sell,"111,000.00"
26,S,WHART,6,DOS,SET,"10,000",12.80,12.20,0.00,0.00%,2,Sell,"128,000.00"


In [6]:
tmp.head().style.hide_index().format(format_dict)

trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
B,BGC,-5,DOS,SET,"10,000",10.00,10.50,0.00,0.00%,1,Buy,"100,000.00"
B,EPG,-5,RD10pct,SET100,"6,000",9.90,10.30,-0.30,-2.83%,2,Buy,"59,400.00"
B,KTC,-11,1L,SET50,"5,000",57.00,59.75,-0.25,-0.42%,0,Buy,"285,000.00"
B,NOBLE,-7,RD15%,SET,"21,000",5.85,6.20,0.05,+0.81%,1,Buy,"122,850.00"
B,PTT,-8,RD05%,SET50,"3,000",36.50,38.50,0.25,+0.65%,1,Buy,"109,500.00"


### End of Tables in the process

### Print to verify before upload file

In [77]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [78]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)

### End of transactional process

In [79]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,EPG,"6,000",9.90,"59,400.00",RD10pct,SET100,2
1,B,STA,"2,500",28.00,"70,000.00",RD15pct,SET100,2
2,S,MCS,"5,000",14.60,"73,000.00",CUT,SET,2
3,S,NER,"9,000",7.30,"65,700.00",CUT,SET,2
4,S,TOP,"2,000",53.25,"106,500.00",5pct,SET50,2
5,S,WHART,"10,000",12.80,"128,000.00",DOS,SET,2


In [80]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    129400.0
S    373200.0
Name: amount, dtype: float64

In [81]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,EPG,"6,000",9.90,"59,400.00",RD10pct,SET100,2
1,B,STA,"2,500",28.00,"70,000.00",RD15pct,SET100,2


In [82]:
cash = 271_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(271000, 129400.0, 141600.0)

In [83]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
2,S,MCS,"5,000",14.60,"73,000.00",CUT,SET,2
3,S,NER,"9,000",7.30,"65,700.00",CUT,SET,2
4,S,TOP,"2,000",53.25,"106,500.00",5pct,SET50,2
5,S,WHART,"10,000",12.80,"128,000.00",DOS,SET,2


In [84]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(271000, 373200.0, 644200.0)

### After call ord-log (must call everytime that orders change)

In [85]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [86]:
show(df[['trans','name','spd','reason','qty','target','amount','current','active']])

trans,name,spd,reason,qty,target,amount,current,active


In [87]:
mb = (df.trans == 'Buy') & (df.spd >= -5)
df[mb].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
1,B,EPG,-5,RD10pct,SET100,"6,000",9.90,10.30,-0.30,-2.83%,2,Buy,"59,400.00"
9,B,ROJNA,-3,6L,SET,"18,000",6.30,6.45,0.05,+0.78%,1,Buy,"113,400.00"
12,B,STA,-3,RD15pct,SET100,"2,500",28.00,28.75,-0.25,-0.86%,2,Buy,"70,000.00"


In [88]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [89]:
ms = (df.trans == 'Sell') & (df.spd <= 5)
df[ms].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
21,S,MCS,1,CUT,SET,"5,000",14.60,14.50,0.00,0.00%,2,Sell,"73,000.00"
22,S,NER,5,CUT,SET,"9,000",7.30,7.05,-0.05,-0.70%,2,Sell,"65,700.00"
24,S,TOP,2,5pct,SET50,"2,000",53.25,52.75,2.00,+3.94%,2,Sell,"106,500.00"


In [90]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
1,B,EPG,-5,RD10pct,SET100,"6,000",9.90,10.30,-0.30,-2.83%,2,Buy,"59,400.00"
9,B,ROJNA,-3,6L,SET,"18,000",6.30,6.45,0.05,+0.78%,1,Buy,"113,400.00"
12,B,STA,-3,RD15pct,SET100,"2,500",28.00,28.75,-0.25,-0.86%,2,Buy,"70,000.00"
21,S,MCS,1,CUT,SET,"5,000",14.60,14.50,0.00,0.00%,2,Sell,"73,000.00"
22,S,NER,5,CUT,SET,"9,000",7.30,7.05,-0.05,-0.70%,2,Sell,"65,700.00"
24,S,TOP,2,5pct,SET50,"2,000",53.25,52.75,2.00,+3.94%,2,Sell,"106,500.00"


In [91]:
df[mb | ms].shape[0]

6

In [92]:
mask = (df.trade == 'B')
df[mb | ms].nlargest(5, "amount")[["name","trans","amount"]].style.format(format_dict)

,name,trans,amount
9,ROJNA,Buy,"113,400.00"
24,TOP,Sell,"106,500.00"
21,MCS,Sell,"73,000.00"
12,STA,Buy,"70,000.00"
22,NER,Sell,"65,700.00"


### Select source of orders between these two

In [93]:
df_out = df[mb | ms][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent
1,Buy,EPG,"6,000",9.90,2,-5,10.30,-0.30,-2.83%
9,Buy,ROJNA,"18,000",6.30,1,-3,6.45,0.05,+0.78%
12,Buy,STA,"2,500",28.00,2,-3,28.75,-0.25,-0.86%
21,Sell,MCS,"5,000",14.60,2,1,14.50,0.00,0.00%
22,Sell,NER,"9,000",7.30,2,5,7.05,-0.05,-0.70%
24,Sell,TOP,"2,000",53.25,2,2,52.75,2.00,+3.94%


In [94]:
df_out = df[df.active == 2][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent
1,Buy,EPG,"6,000",9.90,2,-5,10.30,-0.30,-2.83%
12,Buy,STA,"2,500",28.00,2,-3,28.75,-0.25,-0.86%
21,Sell,MCS,"5,000",14.60,2,1,14.50,0.00,0.00%
22,Sell,NER,"9,000",7.30,2,5,7.05,-0.05,-0.70%
24,Sell,TOP,"2,000",53.25,2,2,52.75,2.00,+3.94%
27,Sell,WHART,"10,000",12.80,2,8,12.00,-0.20,-1.64%


In [95]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)

### End of Evening Process